# Towards a Conflict Heuristic (DH 2023)

## 08. Novel Ranking Top Percentage

Last updated: 25.04.2023

julian.haeussler[at]tu-darmstadt.de

In [1]:
# imports

import csv
import glob
import os
import pandas as pd
import pickle
import re

In [2]:
# read in lists of lists of VPs

with open('..//Analyseergebnisse//pickled//230116_lst_lists_phrases_Romantik_core_TOKENIZED.pkl', 'rb') as f:
    lst_lists_phrases_Romantik = pickle.load(f)

In [3]:
with open('..//Analyseergebnisse//pickled//230116_lst_lists_phrases_Realismus_TOKENIZED.pkl', 'rb') as f:
    lst_lists_phrases_Realismus = pickle.load(f)

In [4]:
with open('..//Analyseergebnisse//pickled//230116_lst_lists_phrases_Naturalismus_TOKENIZED.pkl', 'rb') as f:
    lst_lists_phrases_Naturalismus = pickle.load(f)

In [5]:
len(lst_lists_phrases_Romantik)

26

In [6]:
len(lst_lists_phrases_Realismus)

56

In [7]:
len(lst_lists_phrases_Naturalismus)

46

In [8]:
# merge three lists

lst_lists_phrases_all = lst_lists_phrases_Romantik.copy()

In [9]:
lst_lists_phrases_all.extend(lst_lists_phrases_Realismus)

In [10]:
lst_lists_phrases_all.extend(lst_lists_phrases_Naturalismus)

In [11]:
len(lst_lists_phrases_all)

128

In [12]:
# read in lists of extreme groups

with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_A_adj.pkl', 'rb') as f:
    lst_VPs_A_adj = pickle.load(f)

In [13]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_A_noun.pkl', 'rb') as f:
    lst_VPs_A_noun = pickle.load(f)

In [14]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_all_bottom_aro_adj.pkl', 'rb') as f:
    lst_VPs_bottom_aro_adj = pickle.load(f)

In [15]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_all_bottom_aro_noun.pkl', 'rb') as f:
    lst_VPs_bottom_aro_noun = pickle.load(f)

In [16]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_all_bottom_conf_anno.pkl', 'rb') as f:
    lst_VPs_bottom_conf_anno = pickle.load(f)

In [17]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_all_bottom_conf_dorn.pkl', 'rb') as f:
    lst_VPs_bottom_conf_dorn = pickle.load(f)

In [18]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_all_bottom_val_adj.pkl', 'rb') as f:
    lst_VPs_bottom_val_adj = pickle.load(f)

In [19]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_all_bottom_val_noun.pkl', 'rb') as f:
    lst_VPs_bottom_val_noun = pickle.load(f)

In [20]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_all_top_aro_adj.pkl', 'rb') as f:
    lst_VPs_top_aro_adj = pickle.load(f)

In [21]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_all_top_aro_noun.pkl', 'rb') as f:
    lst_VPs_top_aro_noun = pickle.load(f)

In [22]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_all_top_conf_anno.pkl', 'rb') as f:
    lst_VPs_top_conf_anno = pickle.load(f)

In [23]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_all_top_conf_dorn.pkl', 'rb') as f:
    lst_VPs_top_conf_dorn = pickle.load(f)

In [24]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_all_top_val_adj.pkl', 'rb') as f:
    lst_VPs_top_val_adj = pickle.load(f)

In [25]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_all_top_val_noun.pkl', 'rb') as f:
    lst_VPs_top_val_noun = pickle.load(f)

In [26]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_B_adj.pkl', 'rb') as f:
    lst_VPs_B_adj = pickle.load(f)

In [27]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_B_noun.pkl', 'rb') as f:
    lst_VPs_B_noun = pickle.load(f)

In [28]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_C_adj.pkl', 'rb') as f:
    lst_VPs_C_adj = pickle.load(f)

In [29]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_C_noun.pkl', 'rb') as f:
    lst_VPs_C_noun = pickle.load(f)

In [30]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_D_adj.pkl', 'rb') as f:
    lst_VPs_D_adj = pickle.load(f)

In [31]:
with open('..//Analyseergebnisse//pickled//groups//230425_lst_VPs_D_noun.pkl', 'rb') as f:
    lst_VPs_D_noun = pickle.load(f)

In [32]:
# create empty novel ranking

df_novels = pd.DataFrame(columns=['novel','year',
                                  'A_adj_abs','A_adj_rel','A_noun_abs','A_noun_rel',
                                  'B_adj_abs','B_adj_rel','B_noun_abs','B_noun_rel',
                                  'C_adj_abs','C_adj_rel','C_noun_abs','C_noun_rel',
                                  'D_adj_abs','D_adj_rel','D_noun_abs','D_noun_rel',
                                  'val_adj_top_abs', 'val_adj_top_rel', 'val_adj_bottom_abs', 'val_adj_bottom_rel',
                                  'val_noun_top_abs', 'val_noun_top_rel', 'val_noun_bottom_abs', 'val_noun_bottom_rel',                                
                                  'aro_adj_top_abs', 'aro_adj_top_rel', 'aro_adj_bottom_abs', 'aro_adj_bottom_rel',                                  
                                  'aro_noun_top_abs', 'aro_noun_top_rel', 'aro_noun_bottom_abs', 'aro_noun_bottom_rel', 
                                  'conf_dorn_top_abs','conf_dorn_top_rel','conf_dorn_bottom_abs','conf_dorn_bottom_rel',
                                  'conf_anno_top_abs','conf_anno_top_rel','conf_anno_bottom_abs','conf_anno_bottom_rel'])

In [33]:
# add novel titles

romane_titel = []

fileList = glob.glob(os.path.join(os.getcwd(), r"C:\Users\Julian\HESSENBOX-DA\Projekte\Konflikte\Daten\Romantik (abgeschlossen)\TXT\Kernkorpus", "*.txt"))

for entry in fileList:
    romane_titel.append(re.search(r"(?<=Kernkorpus\\)(.*)(?=.txt)",entry).group(1))

In [34]:
fileList = glob.glob(os.path.join(os.getcwd(), r"C:\Users\Public\Data\Korpuserstellung\Realismus", "*.txt"))

for entry in fileList:
    romane_titel.append(re.search(r"(?<=Realismus\\)(.*)(?=.txt)",entry).group(1))

In [35]:
fileList = glob.glob(os.path.join(os.getcwd(), r"C:\Users\Public\Data\Korpuserstellung\Naturalismus", "*.txt"))

for entry in fileList:
    romane_titel.append(re.search(r"(?<=Naturalismus\\)(.*)(?=.txt)",entry).group(1))

In [36]:
len(romane_titel)

128

In [37]:
df_novels['novel'] = romane_titel

In [38]:
df_novels.head()

,novel,year,A_adj_abs,A_adj_rel,A_noun_abs,A_noun_rel,B_adj_abs,B_adj_rel,B_noun_abs,B_noun_rel,...,aro_noun_bottom_abs,aro_noun_bottom_rel,conf_dorn_top_abs,conf_dorn_top_rel,conf_dorn_bottom_abs,conf_dorn_bottom_rel,conf_anno_top_abs,conf_anno_top_rel,conf_anno_bottom_abs,conf_anno_bottom_rel
0,"Arnim_Achim_von_Armut,_Reichtum,_Schuld_und_Bu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arnim_Achim_von_Die_Kronenwaechter_Erster_Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arnim_Achim_von_Die_Kronenwaechter_Zweiter_Band,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arnim_Achim_von_Hollins_Liebeleben,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Brentano_Clemens_Godwi_oder_Das_steinerne_Bild...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# add publication year

# ROMANTIK

file = r'C:\Users\Julian\HESSENBOX-DA\Projekte\Konflikte\Daten\Romantik (abgeschlossen)\Romane der Romantik_selection.xlsx'

df_meta_Romantik = pd.read_excel(file)

In [40]:
lst_yrs_Romantik = df_meta_Romantik['Datum Erstdruck'].tolist()

In [41]:
lst_yrs_Romantik[:5]

[1810, 1817, 1812, 1802, 1802]

In [42]:
# REALISMUS

file = r'C:\Users\Public\Data\Korpuserstellung\Metadaten_Korpus_Realismus_Romane_d-Prose.xlsx'

df_meta_Realismus = pd.read_excel(file)

In [43]:
lst_yrs_Realismus = df_meta_Realismus['verwendetes Datum'].tolist()

In [44]:
lst_yrs_Realismus[:5]

[1892, 1889, 1888, 1884, 1897]

In [45]:
# NATURALISMUS

file = r'C:\Users\Public\Data\Korpuserstellung\Metadaten_Korpus_Naturalismus_Romane_d-Prose.xlsx'

df_meta_Naturalismus = pd.read_excel(file)

In [46]:
lst_yrs_Naturalismus = df_meta_Naturalismus['verwendetes Datum'].tolist()

In [47]:
lst_yrs_Naturalismus[:5]

[1895, 1884, 1886, 1886, 1897]

In [48]:
# merge years and add to df

lst_yrs = lst_yrs_Romantik.copy()

In [49]:
lst_yrs.extend(lst_yrs_Realismus)

lst_yrs.extend(lst_yrs_Naturalismus)

In [50]:
len(lst_yrs)

128

In [51]:
df_novels['year'] = lst_yrs

In [52]:
df_novels.head()

,novel,year,A_adj_abs,A_adj_rel,A_noun_abs,A_noun_rel,B_adj_abs,B_adj_rel,B_noun_abs,B_noun_rel,...,aro_noun_bottom_abs,aro_noun_bottom_rel,conf_dorn_top_abs,conf_dorn_top_rel,conf_dorn_bottom_abs,conf_dorn_bottom_rel,conf_anno_top_abs,conf_anno_top_rel,conf_anno_bottom_abs,conf_anno_bottom_rel
0,"Arnim_Achim_von_Armut,_Reichtum,_Schuld_und_Bu...",1810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arnim_Achim_von_Die_Kronenwaechter_Erster_Band,1817,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arnim_Achim_von_Die_Kronenwaechter_Zweiter_Band,1812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arnim_Achim_von_Hollins_Liebeleben,1802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Brentano_Clemens_Godwi_oder_Das_steinerne_Bild...,1802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
df_novels.tail()

,novel,year,A_adj_abs,A_adj_rel,A_noun_abs,A_noun_rel,B_adj_abs,B_adj_rel,B_noun_abs,B_noun_rel,...,aro_noun_bottom_abs,aro_noun_bottom_rel,conf_dorn_top_abs,conf_dorn_top_rel,conf_dorn_bottom_abs,conf_dorn_bottom_rel,conf_anno_top_abs,conf_anno_top_rel,conf_anno_bottom_abs,conf_anno_bottom_rel
123,von_Wolzogen_Ernst_Das_dritte_Geschlecht,1899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,von_Wolzogen_Ernst_Der_Kraft-Mayr,1897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,von_Wolzogen_Ernst_Der_Thronfolger,1892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,von_Wolzogen_Ernst_Die_Erbschleicherinnen,1895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,von_Wolzogen_Ernst_Die_Kinder_der_Excellenz,1888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# fill novel ranking

set_VPs_A_adj = set((tuple(phrase) for phrase in lst_VPs_A_adj))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_A_adj))
    rel_freq = count/len(novel)
    df_novels.at[i,'A_adj_abs'] = count
    df_novels.at[i,'A_adj_rel'] = rel_freq

In [55]:
df_novels.head()

,novel,year,A_adj_abs,A_adj_rel,A_noun_abs,A_noun_rel,B_adj_abs,B_adj_rel,B_noun_abs,B_noun_rel,...,aro_noun_bottom_abs,aro_noun_bottom_rel,conf_dorn_top_abs,conf_dorn_top_rel,conf_dorn_bottom_abs,conf_dorn_bottom_rel,conf_anno_top_abs,conf_anno_top_rel,conf_anno_bottom_abs,conf_anno_bottom_rel
0,"Arnim_Achim_von_Armut,_Reichtum,_Schuld_und_Bu...",1810,2163,0.122877,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arnim_Achim_von_Die_Kronenwaechter_Erster_Band,1817,974,0.088877,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arnim_Achim_von_Die_Kronenwaechter_Zweiter_Band,1812,887,0.089578,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arnim_Achim_von_Hollins_Liebeleben,1802,441,0.159378,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Brentano_Clemens_Godwi_oder_Das_steinerne_Bild...,1802,1845,0.122283,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
set_VPs_A_noun = set((tuple(phrase) for phrase in lst_VPs_A_noun))


for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_A_noun))
    rel_freq = count/len(novel)
    df_novels.at[i,'A_noun_abs'] = count
    df_novels.at[i,'A_noun_rel'] = rel_freq

In [57]:
set_VPs_B_adj = set((tuple(phrase) for phrase in lst_VPs_B_adj))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_B_adj))
    rel_freq = count/len(novel)
    df_novels.at[i,'B_adj_abs'] = count
    df_novels.at[i,'B_adj_rel'] = rel_freq

In [58]:
set_VPs_B_noun = set((tuple(phrase) for phrase in lst_VPs_B_noun))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_B_noun))
    rel_freq = count/len(novel)
    df_novels.at[i,'B_noun_abs'] = count
    df_novels.at[i,'B_noun_rel'] = rel_freq

In [59]:
set_VPs_C_adj = set((tuple(phrase) for phrase in lst_VPs_C_adj))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_C_adj))
    rel_freq = count/len(novel)
    df_novels.at[i,'C_adj_abs'] = count
    df_novels.at[i,'C_adj_rel'] = rel_freq

In [60]:
set_VPs_C_noun = set((tuple(phrase) for phrase in lst_VPs_C_noun))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_C_noun))
    rel_freq = count/len(novel)
    df_novels.at[i,'C_noun_abs'] = count
    df_novels.at[i,'C_noun_rel'] = rel_freq

In [61]:
set_VPs_D_adj = set((tuple(phrase) for phrase in lst_VPs_D_adj))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_D_adj))
    rel_freq = count/len(novel)
    df_novels.at[i,'D_adj_abs'] = count
    df_novels.at[i,'D_adj_rel'] = rel_freq

In [62]:
set_VPs_D_noun = set((tuple(phrase) for phrase in lst_VPs_D_noun))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_D_noun))
    rel_freq = count/len(novel)
    df_novels.at[i,'D_noun_abs'] = count
    df_novels.at[i,'D_noun_rel'] = rel_freq

In [63]:
df_novels.head()

,novel,year,A_adj_abs,A_adj_rel,A_noun_abs,A_noun_rel,B_adj_abs,B_adj_rel,B_noun_abs,B_noun_rel,...,aro_noun_bottom_abs,aro_noun_bottom_rel,conf_dorn_top_abs,conf_dorn_top_rel,conf_dorn_bottom_abs,conf_dorn_bottom_rel,conf_anno_top_abs,conf_anno_top_rel,conf_anno_bottom_abs,conf_anno_bottom_rel
0,"Arnim_Achim_von_Armut,_Reichtum,_Schuld_und_Bu...",1810,2163,0.122877,306,0.017383,27,0.001534,31,0.001761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arnim_Achim_von_Die_Kronenwaechter_Erster_Band,1817,974,0.088877,132,0.012045,24,0.00219,20,0.001825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arnim_Achim_von_Die_Kronenwaechter_Zweiter_Band,1812,887,0.089578,147,0.014845,24,0.002424,21,0.002121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arnim_Achim_von_Hollins_Liebeleben,1802,441,0.159378,37,0.013372,3,0.001084,7,0.00253,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Brentano_Clemens_Godwi_oder_Das_steinerne_Bild...,1802,1845,0.122283,350,0.023197,24,0.001591,48,0.003181,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df_novels['C_adj_abs'].head()

0    7
1    3
2    5
3    1
4    9
Name: C_adj_abs, dtype: object

In [65]:
df_novels['C_adj_rel'].head()

0    0.000398
1    0.000274
2    0.000505
3    0.000361
4    0.000597
Name: C_adj_rel, dtype: object

In [66]:
set_VPs_top_val_adj = set((tuple(phrase) for phrase in lst_VPs_top_val_adj))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_top_val_adj))
    rel_freq = count/len(novel)
    df_novels.at[i,'val_adj_top_abs'] = count
    df_novels.at[i,'val_adj_top_rel'] = rel_freq

In [67]:
set_VPs_bottom_val_adj = set((tuple(phrase) for phrase in lst_VPs_bottom_val_adj))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_bottom_val_adj))
    rel_freq = count/len(novel)
    df_novels.at[i,'val_adj_bottom_abs'] = count
    df_novels.at[i,'val_adj_bottom_rel'] = rel_freq

In [68]:
set_VPs_top_val_noun = set((tuple(phrase) for phrase in lst_VPs_top_val_noun))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_top_val_noun))
    rel_freq = count/len(novel)
    df_novels.at[i,'val_noun_top_abs'] = count
    df_novels.at[i,'val_noun_top_rel'] = rel_freq

In [69]:
set_VPs_bottom_val_noun = set((tuple(phrase) for phrase in lst_VPs_bottom_val_noun))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_bottom_val_noun))
    rel_freq = count/len(novel)
    df_novels.at[i,'val_noun_bottom_abs'] = count
    df_novels.at[i,'val_noun_bottom_rel'] = rel_freq

In [70]:
set_VPs_top_aro_adj = set((tuple(phrase) for phrase in lst_VPs_top_aro_adj))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_top_aro_adj))
    rel_freq = count/len(novel)
    df_novels.at[i,'aro_adj_top_abs'] = count
    df_novels.at[i,'aro_adj_top_rel'] = rel_freq

In [71]:
set_VPs_bottom_aro_adj = set((tuple(phrase) for phrase in lst_VPs_bottom_aro_adj))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_bottom_aro_adj))
    rel_freq = count/len(novel)
    df_novels.at[i,'aro_adj_bottom_abs'] = count
    df_novels.at[i,'aro_adj_bottom_rel'] = rel_freq

In [72]:
set_VPs_top_aro_noun = set((tuple(phrase) for phrase in lst_VPs_top_aro_noun))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_top_aro_noun))
    rel_freq = count/len(novel)
    df_novels.at[i,'aro_noun_top_abs'] = count
    df_novels.at[i,'aro_noun_top_rel'] = rel_freq

In [73]:
set_VPs_bottom_aro_noun = set((tuple(phrase) for phrase in lst_VPs_bottom_aro_noun))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_bottom_aro_noun))
    rel_freq = count/len(novel)
    df_novels.at[i,'aro_noun_bottom_abs'] = count
    df_novels.at[i,'aro_noun_bottom_rel'] = rel_freq

In [74]:
df_novels.head()

,novel,year,A_adj_abs,A_adj_rel,A_noun_abs,A_noun_rel,B_adj_abs,B_adj_rel,B_noun_abs,B_noun_rel,...,aro_noun_bottom_abs,aro_noun_bottom_rel,conf_dorn_top_abs,conf_dorn_top_rel,conf_dorn_bottom_abs,conf_dorn_bottom_rel,conf_anno_top_abs,conf_anno_top_rel,conf_anno_bottom_abs,conf_anno_bottom_rel
0,"Arnim_Achim_von_Armut,_Reichtum,_Schuld_und_Bu...",1810,2163,0.122877,306,0.017383,27,0.001534,31,0.001761,...,1807,0.102653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arnim_Achim_von_Die_Kronenwaechter_Erster_Band,1817,974,0.088877,132,0.012045,24,0.00219,20,0.001825,...,1308,0.119354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arnim_Achim_von_Die_Kronenwaechter_Zweiter_Band,1812,887,0.089578,147,0.014845,24,0.002424,21,0.002121,...,1193,0.120481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arnim_Achim_von_Hollins_Liebeleben,1802,441,0.159378,37,0.013372,3,0.001084,7,0.00253,...,249,0.089989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Brentano_Clemens_Godwi_oder_Das_steinerne_Bild...,1802,1845,0.122283,350,0.023197,24,0.001591,48,0.003181,...,1310,0.086824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
set_VPs_top_conf_dorn = set((tuple(phrase) for phrase in lst_VPs_top_conf_dorn))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_top_conf_dorn))
    rel_freq = count/len(novel)
    df_novels.at[i,'conf_dorn_top_abs'] = count
    df_novels.at[i,'conf_dorn_top_rel'] = rel_freq

In [76]:
set_VPs_bottom_conf_dorn = set((tuple(phrase) for phrase in lst_VPs_bottom_conf_dorn))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_bottom_conf_dorn))
    rel_freq = count/len(novel)
    df_novels.at[i,'conf_dorn_bottom_abs'] = count
    df_novels.at[i,'conf_dorn_bottom_rel'] = rel_freq

In [77]:
set_VPs_top_conf_anno = set((tuple(phrase) for phrase in lst_VPs_top_conf_anno))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_top_conf_anno))
    rel_freq = count/len(novel)
    df_novels.at[i,'conf_anno_top_abs'] = count
    df_novels.at[i,'conf_anno_top_rel'] = rel_freq

In [78]:
set_VPs_bottom_conf_anno = set((tuple(phrase) for phrase in lst_VPs_bottom_conf_anno))

for i in range(0,len(lst_lists_phrases_all)):
    count = 0
    novel = lst_lists_phrases_all[i]
    novel_set = set((tuple(phrase) for phrase in novel))
    count = len(novel_set.intersection(set_VPs_bottom_conf_anno))
    rel_freq = count/len(novel)
    df_novels.at[i,'conf_anno_bottom_abs'] = count
    df_novels.at[i,'conf_anno_bottom_rel'] = rel_freq

In [79]:
df_novels.head()

,novel,year,A_adj_abs,A_adj_rel,A_noun_abs,A_noun_rel,B_adj_abs,B_adj_rel,B_noun_abs,B_noun_rel,...,aro_noun_bottom_abs,aro_noun_bottom_rel,conf_dorn_top_abs,conf_dorn_top_rel,conf_dorn_bottom_abs,conf_dorn_bottom_rel,conf_anno_top_abs,conf_anno_top_rel,conf_anno_bottom_abs,conf_anno_bottom_rel
0,"Arnim_Achim_von_Armut,_Reichtum,_Schuld_und_Bu...",1810,2163,0.122877,306,0.017383,27,0.001534,31,0.001761,...,1807,0.102653,312,0.017724,3811,0.216497,64,0.003636,8734,0.496165
1,Arnim_Achim_von_Die_Kronenwaechter_Erster_Band,1817,974,0.088877,132,0.012045,24,0.00219,20,0.001825,...,1308,0.119354,298,0.027192,1658,0.151291,76,0.006935,4571,0.4171
2,Arnim_Achim_von_Die_Kronenwaechter_Zweiter_Band,1812,887,0.089578,147,0.014845,24,0.002424,21,0.002121,...,1193,0.120481,351,0.035447,1484,0.149869,88,0.008887,3655,0.369117
3,Arnim_Achim_von_Hollins_Liebeleben,1802,441,0.159378,37,0.013372,3,0.001084,7,0.00253,...,249,0.089989,81,0.029274,578,0.20889,8,0.002891,1466,0.529816
4,Brentano_Clemens_Godwi_oder_Das_steinerne_Bild...,1802,1845,0.122283,350,0.023197,24,0.001591,48,0.003181,...,1310,0.086824,219,0.014515,3758,0.249072,50,0.003314,8143,0.5397


In [80]:
df_novels.tail()

,novel,year,A_adj_abs,A_adj_rel,A_noun_abs,A_noun_rel,B_adj_abs,B_adj_rel,B_noun_abs,B_noun_rel,...,aro_noun_bottom_abs,aro_noun_bottom_rel,conf_dorn_top_abs,conf_dorn_top_rel,conf_dorn_bottom_abs,conf_dorn_bottom_rel,conf_anno_top_abs,conf_anno_top_rel,conf_anno_bottom_abs,conf_anno_bottom_rel
123,von_Wolzogen_Ernst_Das_dritte_Geschlecht,1899,7,0.001164,11,0.001829,43,0.00715,236,0.039242,...,391,0.065015,336,0.05587,809,0.134519,512,0.085135,88,0.014633
124,von_Wolzogen_Ernst_Der_Kraft-Mayr,1897,11,0.001063,15,0.001449,92,0.008887,315,0.030429,...,784,0.075734,627,0.060568,1172,0.113215,1000,0.0966,118,0.011399
125,von_Wolzogen_Ernst_Der_Thronfolger,1892,9,0.001215,15,0.002025,57,0.007694,268,0.036177,...,419,0.05656,532,0.071814,692,0.093413,521,0.070329,94,0.012689
126,von_Wolzogen_Ernst_Die_Erbschleicherinnen,1895,14,0.001247,17,0.001514,142,0.012644,300,0.026712,...,1067,0.095005,642,0.057163,1047,0.093224,951,0.084676,112,0.009972
127,von_Wolzogen_Ernst_Die_Kinder_der_Excellenz,1888,4,0.000772,8,0.001544,50,0.009653,168,0.032432,...,314,0.060618,287,0.055405,610,0.117761,365,0.070463,53,0.010232


In [81]:
# save

df_novels.to_csv('..//Analyseergebnisse//230425_df_novel_ranking_romanticism_realism_naturalism_WSETS.csv', encoding='utf-8-sig') 

In [82]:
df_novels.to_pickle('..//Analyseergebnisse//230425_df_novel_ranking_romanticism_realism_naturalism_WSETS.pkl')